### Initial playground for domain adaptation and moving into active learning

* Paul Blankley 

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch 
import pyro

%matplotlib inline

In [6]:
# for loading and batching MNIST dataset
def setup_data_loaders(batch_size=128, use_cuda=False):
    root = './data'
    download = True
    trans = transforms.ToTensor()
    train_set = dset.MNIST(root=root, train=True, transform=trans,
                           download=download)
    test_set = dset.MNIST(root=root, train=False, transform=trans)

    kwargs = {'num_workers': 1, 'pin_memory': use_cuda}
    train_loader = torch.utils.data.DataLoader(dataset=train_set,
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(dataset=test_set,
        batch_size=batch_size, shuffle=False, **kwargs)
    return train_loader, test_loader

train_loader, test_loader = setup_data_loaders(batch_size=64, use_cuda=False)

In [7]:
len(train_loader)

938

In [8]:
# Define the model
class logreg(nn.Module):
    def __init__(self, classes):
        super(logreg, self).__init__()
        self.classes = classes
        self.w = nn.Linear(28*28,classes)
        
    def forward(self, x):
        x = self.w(x.view(-1,1,28*28))
        
        return x.view(-1,self.classes)

losses = []
model = logreg(classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=.01)
itrs = 0

for epoch in range(10):
    for batch_x,batch_y in train_loader:
        batch_x = Variable(batch_x)
        batch_y = Variable(batch_y)
        
        y_pred = model(batch_x)
        loss = loss_func(y_pred, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        itrs+=1
        losses.append(loss.data.numpy()[0])

NameError: name 'nn' is not defined